# Intro to Pandas

In [45]:
import pandas as pd

## Series

In [39]:
city_names = pd.Series(['San Francisco', 'San Jose', 'Sacramento'])
populations = pd.Series([852469, 1015785, 485199])

## DataFrame

In [40]:
cities = pd.DataFrame({ 'city name': city_names, 'population': population })
cities

,city name,population
0,San Francisco,852469
1,San Jose,1015785
2,Sacramento,485199


In [41]:
cities['area'] = pd.Series([46.87, 176.53, 97.92])
cities['density'] = cities['population'] / cities['area']
cities

,city name,population,area,density
0,San Francisco,852469,46.87,18187.945381
1,San Jose,1015785,176.53,5754.177760
2,Sacramento,485199,97.92,4955.055147


## Lambdas

In [42]:
names = cities["city name"]
area = cities['area']
cities["Saint_50_Lambdas"] = names.apply(lambda name: name[0:3] == "San") & area.apply(lambda area: area > 50)
cities

,city name,population,area,density,Saint_50_Lambdas
0,San Francisco,852469,46.87,18187.945381,False
1,San Jose,1015785,176.53,5754.177760,True
2,Sacramento,485199,97.92,4955.055147,False


## List comprehension

In [43]:
def saint_50(city_name, area):
  is_saint = city_name[0:3] == "San"
  is_50 = area > 50
  return is_saint & is_50

# A solution using list comprehension and the zip() function  
cities["Saint_50_List_Compr1"] = [saint_50(city_name, area) for city_name, area in zip(cities['city name'], cities['area'])]
cities

,city name,population,area,density,Saint_50_Lambdas,Saint_50_List_Compr1
0,San Francisco,852469,46.87,18187.945381,False,False
1,San Jose,1015785,176.53,5754.177760,True,True
2,Sacramento,485199,97.92,4955.055147,False,False


In [44]:
# A 3rd solution using two list comprehension
saint = [name[0:3] == "San" for name in cities["city name"]]
area50 = [area > 50 for area in cities['area']]
cities["Saint_50_List_Compr2"] = pd.Series(saint) & pd.Series(area50)
cities

,city name,population,area,density,Saint_50_Lambdas,Saint_50_List_Compr1,Saint_50_List_Compr2
0,San Francisco,852469,46.87,18187.945381,False,False,False
1,San Jose,1015785,176.53,5754.177760,True,True,True
2,Sacramento,485199,97.92,4955.055147,False,False,False
